In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

%matplotlib inline

In [2]:
interval = np.array([-1., 1.])

In [18]:
def get_line(x1, y1, x2, y2):
    A = y2 - y1
    B = x1 - x2
    C = x2*y1 - x1*y2
    
    @np.vectorize
    def line(x):
        return A/-B*x + C/-B
    
    return line, np.array([C, A, B])

In [45]:
n_points = 10
X = np.append(np.ones((n_points, 1)),
              np.random.uniform(*interval, (n_points, 2)),
              axis=1)

In [54]:
support_points = np.random.uniform(*interval, 4)
_, w_true = get_line(*support_points)

y_true = np.sign(np.dot(X, w_true))

In [106]:
def run_PLA(X, y_true):
    w_hat = np.zeros(3)
    
    for n_iter in itertools.count(1, 1):
        clf_result = list(enumerate(np.dot(X, w_hat) == y_true))
        misclassified_idx = [idx for idx, is_correct in clf_result if not is_correct]
        if not misclassified_idx:
            break
        rand_misclf_idx = np.random.choice(misclassified_idx)        
        w_hat = w_hat + y_true[rand_misclf_idx] * X[rand_misclf_idx]
        
        if n_iter % 1000 == 0:
            #print("Iterations so far: {0}".format(n_iter), sep="", end="", flush=False)
            print(misclassified_idx)
        
    return w_hat, n_iter